In [ ]:
import win32com.client as win32
from PIL import ImageGrab
from PIL import Image
import img2pdf
import subprocess
import traceback
import time


class MailDispatcher:

    global file_location, filename, mail
    file_location = 'C:/Data/'
    filename = "Battle GOT.xlsx" # in case you are using an macro based excel update the extension accordingly.

    # Kill all excel process
    subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])

    # This is the function creating PDF from image, skip this if not needed.
    def pdf_maker(image_path, name):
        # storing pdf path
        pdf_path = file_location + name + ".pdf"
        # opening image
        image = Image.open(image_path)
        # converting into chunks using img2pdf
        pdf_bytes = img2pdf.convert(image.filename)
        # opening or creating pdf file
        file = open(pdf_path, "wb")
        # writing pdf files with chunks
        file.write(pdf_bytes)
        # closing image file
        image.close()
        # closing pdf file
        file.close()

    try:
        # This portion of the code deals with triggering of an macro, since we donot have a macro in our excel we are not using this.
        #xlApp = win32.DispatchEx('Excel.Application')
        #wb = xlApp.Workbooks.Open(file_location + filename)
        # Give the proper macro name
        #xlApp.Application.Run("Module.Macroname")
        #wb.Save()
        #wb.Close()
        #xlApp.Application.Quit()

        #xlApp = win32.gencache.EnsureDispatch('Excel.Application') ## use this to trouble shoot
        xlApp = win32.DispatchEx('Excel.Application')
        wb = xlApp.Workbooks.Open(file_location + filename)
        ws = wb.Worksheets('Graph') # name of the sheet in which graph is present.
        win32c = win32.constants
        ws.Range("B2:I16").CopyPicture(Format=win32c.xlBitmap) # give the cells for which you need the image
        img = ImageGrab.grabclipboard()
        img.save(file_location + 'Graph.jpeg',quality=55) # image quality and file size is directly linked
        pdf_maker(file_location + 'Graph.jpeg', 'Graph') # second parameter contains the name with which the pdf will be saved

        # This portion deals with the mail part
        outlook = win32.Dispatch('outlook.application')
        ## Workaround to send the mail from a different mail id-----------  skip this part if you want to send mail from your default mail box
        sendfromAC = None
        for oacc in outlook.Session.Accounts:
            if oacc.SmtpAddress == "abc@mail.com": # Mail id from which to send the mail
                sendfromAC = oacc
                break
        ##----------------------------------------------------------------

        mail = outlook.CreateItem(0)

        ##----------------------------------------------------------------
        if sendfromAC:
            mail._oleobj_.Invoke(*(64209, 0, 8, 0, sendfromAC))  # Msg.SendUsingAccount = oacctouse
        ##----------------------------------------------------------------

        mail.To = 'abc@mail.com'
        mail.Cc = 'abc@mail.com;bcd@mail.com'
        mail.Subject = 'Demo Outlook Automation mail'

        mail.Attachments.Add(file_location + "Graph.pdf")

        attachment1 = mail.Attachments.Add(file_location + 'Graph.jpeg')
        attachment1.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", "Graph")

        mail.HTMLBody = "<HTML lang='en' xmlns='http://www.w3.org/1999/xhtml' xmlns:o='urn:schemas-microsoft-com:office:office'> " \
                        + "<head>" \
                        + "<!--[if gte mso 9]><xml> \
                                <o:OfficeDocumentSettings> \
                                <o:Allowjpeg/> \
                                <o:PixelsPerInch>96</o:PixelsPerInch> \
                                </o:OfficeDocumentSettings> \
                            </xml> \
                            <![endif]-->" \
                        + "</head>" \
                        + "<BODY>"

        mail.HTMLBody = mail.HTMLBody + "<BR>Hello,<b> </b>" \
                        + "<BR><BR> Refer to the image below: </b> ."\
                        + "<html><body><img src='cid:Graph'></body></html>"

        mail.Send()
        wb.Close(True)
        xlApp.Quit()
        time.sleep(10)  # waiting for the mail to be sent from outlook
        print("Mail delivery completed successfully.")

    except Exception as ex:
        subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])
        print("Something went wrong! Detailed error message is given below : ")
        traceback.print_tb(ex.__traceback__)

In [ ]:
import win32com.client as win32
from PIL import ImageGrab
from  xlrd import open_workbook
import os
xlApp = win32.gencache.EnsureDispatch('Excel.Application')
wb = xlApp.Workbooks.Open(r'C:\Buddha\volte.xlsm')
ws = wb.Worksheets('Trend')
xlApp.DisplayAlerts = False

i = 0
for chart in ws.ChartObjects():
    
    print (chart.Name)
    chart.CopyPicture()
        #Create new temporary sheet
    xlApp.ActiveWorkbook.Sheets.Add(After=xlApp.ActiveWorkbook.Sheets(3)).Name="temp_sheet" + str(i)
    temp_sheet = xlApp.ActiveSheet

        #Add chart object to new sheet.
    cht = xlApp.ActiveSheet.ChartObjects().Add(0,0,800, 600)
        #Paste copied chart into new object
    cht.Chart.Paste()
        #Export image
    cht.Chart.Export("chart" + str(i) + ".png")

        #This line is not entirely neccessary since script currently exits without saving
    temp_sheet.Delete()
    i = i+1

xlApp.ActiveWorkbook.Close()
    #Restore default behaviour
xlApp.DisplayAlerts = True



import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'buddhadeb.mondal@ericsson.com'
mail.Subject = 'Message subject'
mail.Body = 'Message body'
mail.HTMLBody = '<h2>HTML Message body</h2>'# this field is optional
mail.Attachments.Add('C:\Buddha\Buddha.jpg')
mail.Send()

In [1]:
import win32com.client as win32
import excel2img
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(r'C:\Buddha\volte.xlsm')
print (wb)
selection = "A1:J1" 
xl_range = excel.Sheets('Trend').Range(selection)
excel.ActiveWorkbook.Sheets.Add(After=excel.ActiveWorkbook.Sheets(1)).Name="image_sheet"
cht = excel.ActiveSheet.ChartObjects().Add(0,0,xl_range.Width, xl_range.Height)
xl_range.CopyPicture()
# add the chart to new sheet
cht.Chart.Paste()
# Export the sheet with the chart to a new file
cht.Chart.Export('C:\\Buddha\\volte11.xlsx')
# Delete the sheet
cht.Delete()
excel.ActiveSheet.Delete()
# Close the book
excel.ActiveWorkbook.Close()

ModuleNotFoundError: No module named 'excel2img'

In [1]:

import win32com.client       # Need pywin32 from pip
from PIL import ImageGrab    # Need PIL as well
import os

excel = win32com.client.Dispatch("Excel.Application")
workbook = excel.ActiveWorkbook

wb_folder = workbook.Path
wb_name = workbook.Name
wb_path = os.path.join(wb_folder, wb_name)

print "Extracting images from %s" % wb_path

image_no = 0

for sheet in workbook.Worksheets:
    for n, shape in enumerate(sheet.Shapes):
        if shape.Name.startswith("Picture"):
            # Some debug output for console
            image_no += 1
            print "---- Image No. %07i ----" % image_no

            # Sequence number the pictures, if there's more than one
            num = "" if n == 0 else "_%03i" % n

            filename = sheet.Name + num + ".jpg"
            file_path = os.path.join (wb_folder, filename)

            print "Saving as %s" % file_path    # Debug output

            shape.Copy() # Copies from Excel to Windows clipboard

            # Use PIL (python imaging library) to save from Windows clipboard
            # to a file
            image = ImageGrab.grabclipboard()
            image.save(file_path,'jpeg')

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Extracting images from %s" % wb_path)? (<ipython-input-1-b50c526df049>, line 12)

In [2]:
import win32com.client as win32
from win32com.client import Dispatch 
#import win32clipboard


xlApp = Dispatch('Excel.Application')
wb = xlApp.Workbooks.Open('C:\\Buddha\\volte.xlsm')
xlApp.Sheets("Trend").Select()

#xlSheet1 = xlApp.Sheets(1)
ws = wb.Worksheets('Trend') # name of the sheet in which graph is present.
xlApp.DisplayAlerts = False

i = 1
#for sheet in wb.Worksheets:
#    for chartObject in sheet.ChartObjects():
        # print(sheet.Name + ':' + chartObject.Name)
for chart in ws.ChartObjects():

    print (chart.Name)
   
    
    
    xlApp.ActiveWorkbook.Sheets.Add(After=xlApp.ActiveWorkbook.Sheets('Trend')).Name="sheet1"+ str(i)
    temp_sheet = xlApp.ActiveSheet
    print(ws.ChartObjects().Width)
    print(ws.ChartObjects().Left)
    print(ws.ChartObjects().Top)
    #cht = ws.ChartObjects()(chart.Name)

    cht = xlApp.ActiveSheet.ChartObjects().Add(ws.ChartObjects().Left,ws.ChartObjects().Top,ws.ChartObjects().Width,ws.ChartObjects().Height)
   # cht.Chart.Copy
    chart.CopyPicture()
    #xlApp.DisplayAlerts = 
    cht.Chart.Paste()
    #Export image
    cht.Chart.Export("C:\\Buddha\\chart" + str(i) + ".png")
    #temp_sheet.Delete()
   
    temp_sheet.Delete()
    i += 1
    
wb.Close()




com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', "Sorry, we couldn't find C:\\Buddha\\volte.xlsm. Is it possible it was moved, renamed or deleted?", 'xlmain11.chm', 0, -2146827284), None)

In [4]:
import pandas as pd
import win32com.client as win32
import os
import glob
from tkinter import filedialog
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.image import MIMEImage
INPATH=filedialog.askdirectory()
files=glob.glob(INPATH + "/*",recursive =True)

html_2100="""<p><h4 style="font-size:15px;"></h4></p>
			"""
html_2="""</body>
			</html>"""
html_1=""

outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To='sumer.singh.rathore@ericsson.com'
mail.Subject = 'ON AIR'+'Site-'

d={}
d1={}
c=0
i=1
for w in files:     
	if"chart" in w:
		s="a_"+str(i)
		d1[s]=w
		d.update(d1)
		if c==0:
			html_1=html_1+html_2100
			c=c+1
		s="a_"+str(i)
		
		#Mail Body
		path=w        
	
		html = """
		  <img src="cid:{}" alt="Logo" style="width:50px;height:10px;"><br>
		  """.format(s)
		html_1= html_1+html  
		i=i+1
   


html_all=html_1+html_2

for k in d:
	if d[k].endswith(".png"):
		attachment = mail.Attachments.Add(d[k])
		attachment.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", k)

mail.HTMLBody=html_all
mail.Display(True)

In [2]:
import os
import sys
import win32com.client as win32
import base64

def Emailer(text, subject, recipient):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Subject = subject
   # mail.HtmlBody = html_pages + <br><img src="C:\Buddha\chart1.png">
    mail.Attachments.Add('C:\Buddha\chart1.png')
  
    mail.Display(True)

Emailer('hello, So here we need to fetch data from an excel file' , 'Wow it works' , 'hello@test.com')


com_error: (-2147352567, 'Exception occurred.', (4096, 'Microsoft Outlook', 'Cannot find this file. Verify the path and file name are correct.', None, 0, -2147024894), None)

In [3]:
from pptx import Presentation
from pptx.util import Inches

img_path1 = 'C:\Buddha\chart1.png'
img_path2 = 'C:\Buddha\chart2.png'
img_path3 = 'C:\Buddha\chart3.png'
img_path4 = 'C:\Buddha\chart4.png'

prs = Presentation()
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)
left = top = Inches(1)
pic = slide.shapes.add_picture(img_path1, left, top)

blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)
left = top = Inches(1)
pic = slide.shapes.add_picture(img_path2, left, top)

blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)
left = top = Inches(1)
pic = slide.shapes.add_picture(img_path3, left, top)

blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)
left = top = Inches(1)
pic = slide.shapes.add_picture(img_path4, left, top)

prs.save(r'C:\Buddha\test.pptx')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Buddha\\chart1.png'

In [ ]:
import win32com.client as win32
from win32com.client import Dispatch 
#import win32clipboard


xlApp = Dispatch('Excel.Application')
wb = xlApp.Workbooks.Open(r'C:\Users\emonbud\OneDrive - Ericsson AB\volte.xlsm')
xlApp.Sheets("Trend").Select()

#xlSheet1 = xlApp.Sheets(1)
ws = wb.Worksheets('Trend') # name of the sheet in which graph is present.
xlApp.DisplayAlerts = False

i = 1
#for sheet in wb.Worksheets:
#    for chartObject in sheet.ChartObjects():
        # print(sheet.Name + ':' + chartObject.Name)
for chart in ws.ChartObjects():

    print (chart.Name)
   
    
    
    xlApp.ActiveWorkbook.Sheets.Add(After=xlApp.ActiveWorkbook.Sheets('Trend')).Name="sheet1"+ str(i)
    temp_sheet = xlApp.ActiveSheet
    print(ws.ChartObjects().Width)
    print(ws.ChartObjects().Left)
    print(ws.ChartObjects().Top)
    #cht = ws.ChartObjects()(chart.Name)

    cht = xlApp.ActiveSheet.ChartObjects().Add(ws.ChartObjects().Left,ws.ChartObjects().Top,ws.ChartObjects().Width,ws.ChartObjects().Height)
   # cht.Chart.Copy
    chart.CopyPicture()
    #xlApp.DisplayAlerts = 
    cht.Chart.Paste()
    #Export image
    cht.Chart.Export("C:\\Buddha\\chart" + str(i) + ".png")
    #temp_sheet.Delete()
   
    temp_sheet.Delete()
    i += 1
    
wb.Close()

import pandas as pd
import win32com.client as win32
import os
import glob
from tkinter import filedialog
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.image import MIMEImage
INPATH=filedialog.askdirectory()
files=glob.glob(INPATH + "/*",recursive =True)

html_2100="""<p><h4 style="font-size:15px;"></h4></p>
			"""
html_2="""</body>
			</html>"""
html_1=""

outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To='buddhadeb.mondal@ericsson.com'
mail.Subject = 'AT & T Project - Pick Chart from Excel to Outlook'

d={}
d1={}
c=0
i=1
for w in files:     
	if"chart" in w:
		s="a_"+str(i)
		d1[s]=w
		d.update(d1)
		if c==0:
			html_1=html_1+html_2100
			c=c+1
		s="a_"+str(i)
		
		#Mail Body
		path=w        
	
		html = """
		  <img src="cid:{}" alt="Logo" style="width:50px;height:10px;"><br>
		  """.format(s)
		html_1= html_1+html  
		i=i+1
   


html_all=html_1+html_2

for k in d:
	if d[k].endswith(".png"):
		attachment = mail.Attachments.Add(d[k])
		attachment.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", k)

mail.HTMLBody=html_all
mail.Display(True)